# SENTINEL 2 LEVEL 3A DATA  ACQUISITION

In [ ]:
#! pip install eodag

In [ ]:
from eodag import EODataAccessGateway
# Import necessary packages
import os 
import cv2
from datetime import date
import json
import itertools
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelhub import pixel_to_utm, utm_to_pixel
import utm
import pandas as pd
from shapely.geometry import box
from rasterio.mask import mask

In [ ]:
dag = EODataAccessGateway()

In [ ]:
dag.set_preferred_provider("theia")

In [ ]:
import geopandas as gpd

In [ ]:
#aoi_path = "../data/Andramasina_trees_location_2.geojson"
#Set the path to data
aoi_dir = "./aoi"
#aoi_file = "fkkl_rwy.geojson"
aoi_file = "Andramasina_trees_location_2.geojson"
aoi_path = os.path.join(aoi_dir , aoi_file ) 

In [ ]:
aoi = gpd.read_file(aoi_path)


In [ ]:
values = []
for i in range(len(aoi.bounds.iloc[0])):
    values.append(aoi.bounds.iloc[0][i])
keys = ["lonmin", "latmin", "lonmax", "latmax"]  
geometry = dict(zip(keys, values))

In [ ]:
geometry

In [ ]:
# ! eodag list -p theia

In [ ]:
search_criteria = {
    "productType": "S2_MSI_L3A_WASP",
    "start": "2021-04-01",
    "end": "2021-04-30",
    #"geom": {"lonmin": 1, "latmin": 43, "lonmax": 2, "latmax": 44},
    "geom": geometry, 
}

In [ ]:
search_results, total_count = dag.search(**search_criteria)

In [ ]:
print(f"Got {len(search_results)} products and an estimated total number of {total_count} products.")

In [ ]:
#product_paths = dag.download_all(search_results)

In [ ]:
pwd

In [ ]:
image_folder = "./data/SENTINEL2X_20210401-000000-000_L3A_T38KQD_D/SENTINEL2X_20210401-000000-000_L3A_T38KQD_C_V2-3"

In [ ]:
image_files = [im for im in os.listdir(image_folder) if im[-4:] == ".tif"]

In [ ]:
print(image_files)

In [ ]:
selected_file = [im for im in image_files if (im[-6:] in ["B2.tif", "B3.tif", "B4.tif", "B8.tif"])]

In [ ]:
print(selected_file)

In [ ]:
def get_metadata(image_folder):
    
    image_files = [im for im in os.listdir(image_folder) if im[-4:] == ".tif"]
    
    selected_file = [im for im in image_files if (im[-6:] in ["B2.tif", "B3.tif", "B4.tif", "B8.tif"])]
    
    with rasterio.open(f"{image_folder}/{selected_file[0]}") as infile:
        
        metadata = infile.meta.copy()
        
    print(len(selected_file))
            
    return metadata

In [ ]:
import os
meta = get_metadata(image_folder)

In [ ]:
meta

In [ ]:
def get_band(image_folder, band):
    
    image_files = [im for im in os.listdir(image_folder) if im[-4:] == ".tif"]
    
    selected_file = [im for im in image_files if (im[-6:] in ["B2.tif", "B3.tif", "B4.tif", "B8.tif"])]
    
    with rasterio.open(f"{image_folder}/{selected_file[band]}") as infile:
        
        img = infile.read(1)
            
    return img

In [ ]:
band_list = ["B02","B03","B04","B08"]

band_dict = {}

for i, band in enumerate(band_list):
    
    band_dict[band] = get_band(image_folder, i)

In [ ]:
# Set directory for downloading the quad tiles to
out_meta = get_metadata(image_folder)

#rgbnir_out = './raster_process/rgbnir.tif'
bgrnir_out = './data/Andramasina_trees_location_2_theia_05_2022.tif'

out_meta.update({"count": 4})
out_meta.update({'driver':'GTiff'})
#out_meta.update({'crs': 3857})
print(out_meta)

bgrnir = np.dstack([band_dict["B04"], band_dict["B03"], band_dict["B02"],band_dict["B08"]])
# We stack this bands on the following order to match planet NICFI data  band  stack
# so the band arragement are : BGRNIR
bgrnir = bgrnir.transpose(2, 0 ,1)
with rasterio.open(bgrnir_out , "w", **out_meta) as dest:
    dest.write(bgrnir)

<hr/> 

## 6-Clib the image to our AOI

**Note** : 
> After creating the image, the next step is to  and clip + 100m to get the image for processi

In [ ]:
def get_bounds_of_AoI(obj_aoi, src_crs):
    
    aoi = gpd.read_file(obj_aoi)
    
    bounds = aoi.total_bounds
    
    offset = 1/120  #100m in degree

    # WGS84 coordinates
    minx, miny = bounds[0]-offset, bounds[1]-offset
    maxx, maxy = bounds[2]+offset, bounds[3]+offset

    bbox = box(minx, miny, maxx, maxy)
    
    print(bbox)

    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs="EPSG:4326")

    #picking up the coordinate system of the image:
    #crs=src.crs.to_epsg()
    geo = geo.to_crs(crs=src_crs) #src.crs.to_epsg())

    coords = getFeatures(geo)
    
    return coords

    print(coords)

In [ ]:
def clip_raster_with_bounds(in_tif, out_tif, coords):

    #load the mosaided file
    data = rasterio.open(in_tif)

    out_img, out_transform = rasterio.mask.mask(dataset=data, shapes=coords, crop=True)

    # Copy the metadata
    out_meta = data.meta.copy()

    # Parse EPSG code
    epsg_code = int(data.crs['init'][5:])

    out_meta.update({"driver": "GTiff",
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform,
            "crs": epsg_code} #pycrs.parse.from_epsg_code(epsg_code).to_proj4()}
            )

    with rasterio.open(out_tif, "w", **out_meta) as dest:
        dest.write(out_img)

    print(out_tif)

In [ ]:
def get_src_crs(prototype_tif):
    
    with rasterio.open(prototype_tif, 'r') as test:
        src_crs = test.crs.to_epsg()
        
    return src_crs

In [ ]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
# Set directory for downloading the quad tiles to
root_dir= './data/'
sentinel_clip_dir = os.path.join(root_dir,'Andramasina_trees_location_2_L3A/') # <= customized to month/year of interest
if not os.path.exists(sentinel_clip_dir):
    os.makedirs(sentinel_clip_dir)
    
image_file_out = os.path.join(sentinel_clip_dir,'Andramasina_trees_location_2_L3A_clip.tif')

src_crs = get_src_crs(bgrnir_out)
coords = get_bounds_of_AoI(aoi_path ,src_crs)
clip_raster_with_bounds(bgrnir_out,image_file_out, coords)